In [1]:
# data
from total_export_data import *

# data management
import pandas as pd
import numpy as np
import networkx as nx
import operator
import matplotlib.pyplot as plt
from networkx.algorithms.centrality import eigenvector_centrality
from networkx.algorithms.centrality import degree_centrality
from networkx.algorithms.cluster import clustering

#Visualization
import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
df_final = total_export()

/Users/phuongpham/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning:

Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.



>>> Total export == total import ... True


In [3]:
df_final.head()

,location_code,partner_code,export_value,import_value,export_portion,import_portion
0,ABW,ANS,0,128195804,0.000000e+00,4.408608e-06
1,ABW,ARG,0,1626391,0.000000e+00,5.593101e-08
2,ABW,AUT,142117,803973,4.887353e-09,2.764835e-08
3,ABW,AZE,0,4115,0.000000e+00,1.415134e-10
4,ABW,BEL,325434,2846830,1.119156e-08,9.790148e-08


In [4]:
df_final[(df_final['location_code'] == 'CHN') & (df_final['partner_code'] == 'AGO')]

,location_code,partner_code,export_value,import_value,export_portion,import_portion
5784,CHN,AGO,1698636046,13269301448,0.000058,0.000456


In [5]:
# Get the top 5% weights 
top_export = df_final[df_final['export_portion'] > np.quantile(df_final['export_portion'],1-5/100)][['location_code', 'partner_code', 'export_portion']]

In [6]:
top_export.head()

,location_code,partner_code,export_portion
231,AGO,CHN,0.000456
320,AGO,USA,0.000103
673,ANS,AUS,0.000145
674,ANS,AUT,0.000268
682,ANS,CHL,0.000094


In [31]:
# Get the top 5% import
top_import = df_final[df_final['import_portion'] > np.quantile(df_final['import_portion'],1-5/100)][['partner_code', 'location_code', 'import_portion']]

In [32]:
top_import.head()

,partner_code,location_code,import_portion
231,CHN,AGO,0.000058
295,PRT,AGO,0.000078
468,ITA,ALB,0.000084
673,AUS,ANS,0.000522
674,AUT,ANS,0.000196


In [33]:
# Get the edges
export_edges = top_export.values
import_edges = top_import.values

In [34]:
## Build a graph with directed edges going from country which exports to country which imports
network = nx.DiGraph()

In [35]:
network.add_weighted_edges_from(import_edges)

In [36]:
network.add_weighted_edges_from(export_edges)

In [37]:
# # Betweenness centrality
# betweenness = betweenness_centrality(network, weight = 'weight')
# # Find the highest betweenness centrality countries
# highest_betweenness = sorted(betweenness.items(), key=operator.itemgetter(1), reverse = True)

In [38]:
# Degree centrality
degrees_cen = degree_centrality(network)

In [39]:
# Eigenvector centrality
eigenvector = eigenvector_centrality(network, weight = 'weight', max_iter = 1000)

In [40]:
highest_eigenvector = sorted(eigenvector.items(), key=operator.itemgetter(1), reverse = True)
highest_eigenvector[:10]

[('USA', 0.6674181953885323),
 ('CAN', 0.31943270028080806),
 ('MEX', 0.2989400941028825),
 ('CHN', 0.2974727459652848),
 ('DEU', 0.21925261311912442),
 ('HKG', 0.2014351748844698),
 ('GBR', 0.1702977434447148),
 ('JPN', 0.1686336447994271),
 ('FRA', 0.14881436356115735),
 ('NLD', 0.13383170084988003)]

In [41]:
# Position
pos = nx.random_layout(network, seed = 3)
# pos = country_geo[['Alpha-3 code', 'Latitude (average)', 'Longitude (average)']].set_index('Alpha-3 code').T.to_dict('list')


In [42]:
def plot_node_edge(network, attribute, attribute_name, scale = 10):   
    edge_trace = go.Scatter(
        x=[],
        y=[],
        line=dict(width=0.5,color='#888'),
        opacity = 0.3,
        hoverinfo='none',
        mode='lines')
    
    for edge in network.edges():
        start, dest = edge
        if dest == 'ANS' or start == 'ANS': #Undeclared
            continue 
        x0, y0 = pos[start]
        x1, y1 = pos[dest]
        edge_trace['x'] += tuple([x0, x1, None])
        edge_trace['y'] += tuple([y0, y1, None])
        
    node_trace = go.Scatter(
        x=[],
        y=[],
        text=[],
        textposition = 'top center',
        mode='markers+text',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=[],
            colorbar=dict(
                thickness=15,
                title=attribute_name,
                xanchor='left',
                titleside='right'
            ),
            line=dict(width=2)))
    
    for node in network.nodes():
        if node == 'ANS': #Undeclared
            continue
        x, y = pos[node]
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])
        node_trace['marker']['size']+= tuple([attribute[node]*scale])
        node_trace['marker']['color']+=tuple([attribute[node]])
        node_trace['text']+=tuple([node])
    return edge_trace, node_trace

In [43]:
edge_trace, node_trace = plot_node_edge(network, eigenvector, 'Eigenvector centrality', scale = 100)

In [44]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>International Trade Network in 2016',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename='networkx')

In [45]:
d_edge_trace, d_node_trace = plot_node_edge(network, degrees_cen, 'Degree centrality', scale = 30)

In [46]:
fig = go.Figure(data=[d_edge_trace, d_node_trace],
             layout=go.Layout(
                title='<br>International Trade Network in 2016',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename='networkx')

In [60]:
clusters = clustering(network)

In [69]:
c_edge_trace, c_node_trace = plot_node_edge(network, clusters, 'Clustering', scale = 15)

In [70]:
fig = go.Figure(data=[c_edge_trace, c_node_trace],
             layout=go.Layout(
                title='<br>International Trade Network in 2016',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename='networkx')